# 구조화 문장에서 사고원인 추출

### 맞춤법 수정 결과 검증 prompt 준비

In [60]:
import pandas as pd
from utils import load_ollama, run_ollama, parallel_process_n
import re
import json
from tqdm import tqdm
tqdm.pandas()

test_df = pd.read_csv('../data/test_03.csv')
train_df = pd.read_csv('../data/train_03.csv')

test_ce_df = test_df[test_df['category_exists']=="Y"]
train_ce_df = train_df[train_df['category_exists']=="Y"]

### json 형식 검증

In [61]:
llm_response = """
```json
{
  "발생 배경": "test",
  "사고 종류": "test",
  "사고 원인": "test"
}
```
"""

def is_valid_json_reason(text):
    """
    주어진 텍스트 내의 ```json 코드 블록 내부 내용이 유효한 JSON 형식이며,
    그 JSON 데이터에 '사고 원인' 키가 존재하면 True, 그렇지 않으면 False를 반환합니다.
    """
    # 정규표현식을 사용하여 ```json과 ``` 사이의 내용 추출
    match = re.search(r"```json(.*?)```", text, re.DOTALL)
    if match:
        json_str = match.group(1).strip()  # 양쪽 공백 제거
        try:
            data = json.loads(json_str)
            # JSON 데이터가 딕셔너리이고 '사고 원인' 키가 존재하는지 확인
            if isinstance(data, dict) and all(key in data and isinstance(data[key], str) for key in ["사고 원인", "발생 배경", "사고 종류"]):
                return True
            else:
                return False
        except json.JSONDecodeError:
            return False
    return False
    
print(is_valid_json_reason(llm_response))

True


In [62]:
test_ce_df['valid_json'] = test_ce_df['structured_json'].apply(lambda x: is_valid_json_reason(x))
train_ce_df['valid_json'] = train_ce_df['structured_json'].apply(lambda x: is_valid_json_reason(x))

/var/folders/5h/qkt17_v93197z8r0k03p5m2c0000gn/T/ipykernel_2414/231922822.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_ce_df['valid_json'] = test_ce_df['structured_json'].apply(lambda x: is_valid_json_reason(x))
/var/folders/5h/qkt17_v93197z8r0k03p5m2c0000gn/T/ipykernel_2414/231922822.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_ce_df['valid_json'] = train_ce_df['structured_json'].apply(lambda x: is_valid_json_reason(x))


### 잘못된 형식 있는지 확인

In [63]:
train_ce_df[train_ce_df['valid_json']==False]['structured_json'].tolist()

['```json\n{\n  "발생 배경": "지하 5층 기계실에서 냉각수 배관 설치 작업을 진행 중",\n  "사고 종류": "자재 관련 물체에 의한 사고 (발판 낙하)",\n  "사고 원인": "렌탈 장비로 내리던 중 렌탈 장비로 렌탈 장비로 렌탈 장비로 렌탈 장비로 렌탈 장비로 렌탈 장비로 렌탈 장비로 렌탈 장비로 렌탈 장비로 렌탈 장비로 렌탈 장비로 렌탈 장비로 렌탈 장비로 렌탈 장비로 렌탈 장비로 렌탈 장비로 렌탈 장비로 렌탈 장비로 렌탈 장비로 렌탈 장비로 렌탈 장비로 렌탈 장비로 렌탈 장비로 렌탈 장비로 렌탈 장비로 렌탈 장비로 렌탈 장비로 렌탈 장비로 렌탈 장비로 렌탈 장비로 렌탈 장비',
 '```json\n{\n  "발생 배경": "운반작업 중",\n  "사고 종류": "비계 관련 넘어짐 (미끄러짐)",\n  "사고 원인": "미끄러운 곳을 이용하여 발판으로 사용, 미끄러운 곳을 이용하여 발판으로 사용, 미끄러운 곳을 이용하여 발판으로 사용, 미끄러운 곳을 이용하여 발판으로 사용, 미끄러운 곳을 이용하여 발판으로 사용, 미끄러운 곳을 이용하여 발판으로 사용, 미끄러운 곳을 이용하여 발판으로 사용, 미끄러운 곳을 이용하여 발판으로 사용, 미끄러운 곳을 이용하여 발판으로 사용, 미끄러운 곳을 이용하여 발판으로 사용, 미끄러운 곳을 이용하여 발판으로 사용, 미끄러운 곳을 이용하여 발판으로 사용, 미끄러운 곳을 이용하여 발판으로 사용, 미끄러운 곳을 이용하여 발판으로 사용, 미끄러운 곳을 이용하여 발판으로 사용, 미끄러운 곳을 이용하여 발판으로 사용, 미끄러운 곳을 이용하여 발판으로 사용, 미끄러운 곳을',
 '```json\n{\n  "발생 배경": "마감 작업 중 고소 작업대 등 관련 떨어짐 사고",\n  "사고 종류": "고소 작업대 등 관련 떨어짐 사고",\n  "사고 원인": [\n    "근로자의 불안전한 행동 : 작업 종료 시점 빠른 작업을 위하여, 렌탈 이동을 하지 않고 무리한 작업(렌탈문을 열어 팔을 뻗어 작업)",\n    "안전 

In [64]:
test_ce_df[test_ce_df['valid_json']==False]

,id,description,correct_description_prompt,correct_description,category_exists,structuring_prompt,structured_json,valid_json


### ~~적절한 문장 형식 나올 때까지 모델 응답 요청~~
### 문장 형식 제대로 되어도 같은 단어 반복 오류 -> 삭제

In [65]:
# chain = load_ollama(model="gemma3:4b", temperature=1.0)

# def find_proper_form(row):
#     chain = load_ollama(model="gemma3:4b", temperature=1.0, top_p=0.9, num_predict=256)
#     print(row.id)
#     answer = ''
#     while not is_valid_json_reason(answer):
#         answer = run_ollama(row['structuring_prompt'], chain)
#     return answer

# if len(test_ce_df[test_ce_df['valid_json']==False]) != 0:
#     # test_ce_df.loc[test_ce_df['valid_json']==False, 'structured_description'] = test_ce_df[test_ce_df['valid_json']==False].progress_apply(lambda x: find_proper_form(x), axis=1)
#     for idx, row in tqdm(test_ce_df[test_ce_df['valid_json'] == False].iterrows()):
#         test_ce_df.loc[idx, 'structured_description'] = find_proper_form(row)
#         test_ce_df.loc[idx, 'valid_json'] = True
# if len(train_ce_df[train_ce_df['valid_json']==False]) != 0:
#     # train_ce_df.loc[train_ce_df['valid_json']==False, 'structured_description'] = train_ce_df[train_ce_df['valid_json']==False].progress_apply(lambda x: find_proper_form(x), axis=1)
#     # 'valid_json' 컬럼 값이 False인 행들만 선택하여 반복 처리
#     for idx, row in tqdm(train_ce_df[train_ce_df['valid_json'] == False].iterrows()):
#         train_ce_df.loc[idx, 'structured_description'] = find_proper_form(row)
#         train_ce_df.loc[idx, 'valid_json'] = True

In [66]:
train_ce_df[train_ce_df['valid_json']==False].index

Index([603, 6058, 7850, 9282, 9685, 9861, 12790, 15830, 15896, 16742, 17845,
       18697],
      dtype='int64')

In [67]:
# 그래도 안되는 행 삭제
train_df = train_df.drop(train_ce_df[train_ce_df['valid_json']==False].index)
test_df = test_df.drop(test_ce_df[test_ce_df['valid_json']==False].index)

train_ce_df = train_ce_df.drop(train_ce_df[train_ce_df['valid_json']==False].index)
test_ce_df = test_ce_df.drop(test_ce_df[test_ce_df['valid_json']==False].index)

### 구조화 설명문 추출

In [68]:
def make_structured_desription(json_text):
    # 정규표현식을 사용하여 ```json과 ``` 사이의 내용을 추출
    if type(json_text) != str:
        return json_text
    match = re.search(r"```json(.*?)```", json_text, re.DOTALL)
    json_str = match.group(1).strip()  # 양쪽 공백 제거
    data = json.loads(json_str)
    text = f"발생 배경: {data['발생 배경']}, 사고 종류: {data['사고 종류']}, 사고 원인: {data['사고 원인']}"
    return text

test_df['structured_description'] = test_df['structured_json'].apply(make_structured_desription)
train_df['structured_description'] = train_df['structured_json'].apply(make_structured_desription)

In [69]:
test_df.loc[test_df['structured_description'].isna(), 'structured_description'] = test_df.loc[test_df['structured_description'].isna(), 'correct_description']
train_df.loc[train_df['structured_description'].isna(), 'structured_description'] = train_df.loc[train_df['structured_description'].isna(), 'correct_description']

### 구조화 설명문 및 사고원인 추출

In [70]:
def extract_json(text):
    # 정규표현식을 사용하여 ```json과 ``` 사이의 내용을 추출
    match = re.search(r"```json(.*?)```", text, re.DOTALL)
    json_str = match.group(1).strip()  # 양쪽 공백 제거
    data = json.loads(json_str)         # JSON 문자열을 딕셔너리로 변환
    return data.get("사고 원인")         # '사고 원인' 키의 값을 반환

test_df.loc[test_ce_df.index, 'reason'] = test_ce_df['structured_json'].apply(lambda x: extract_json(x))
train_df.loc[train_ce_df.index, 'reason'] = train_ce_df['structured_json'].apply(lambda x: extract_json(x))

In [71]:
# 사고 종류가 없는 데이터는 구조화 못했음. '사고원인' 텍스트 그대로 사용.
test_df.loc[test_df['category_exists']=="N", 'reason'] = test_df.loc[test_df['category_exists']=="N", 'correct_description']
train_df.loc[train_df['category_exists']=="N", 'reason'] = train_df.loc[train_df['category_exists']=="N", 'correct_description']

In [72]:
train_df = train_df[~train_df['reason'].isna()]
test_df = test_df[~test_df['reason'].isna()]

In [73]:
test_df.to_csv('../data/test_04.csv', index=False)
train_df.to_csv('../data/train_04.csv', index=False)